# Examples of scheduled circuit  (circuit with duration)

## Manual construction of T2 experiment

In [1]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(1, 1, name="t2_experiment")
qc.h(0)
qc.delay(100, 0, unit='ns')
qc.h(0)
qc.measure(0, 0)
qc.draw()

┌───┐┌───────┐┌───┐┌─┐
q_0: ┤ H ├┤ DELAY ├┤ H ├┤M├
     └───┘└───────┘└───┘└╥┘
c_0: ════════════════════╩═

## Schedule Bell experiment with scheduling passes

In [2]:
from qiskit import transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.transpiler.passes.scheduling.asap import ASAPSchedule
from qiskit.transpiler.passes.scheduling.alap import ALAPSchedule

In [3]:
from qiskit.test.mock.backends import FakeParis
backend = FakeParis()

In [4]:
qc = QuantumCircuit(2, name="bell")
qc.h(0)
qc.delay(1000, 1, unit='ns')
qc.cx(0,1)
qc.draw()

┌───┐       
q_0: ──┤ H ├────■──
     ┌─┴───┴─┐┌─┴─┐
q_1: ┤ DELAY ├┤ X ├
     └───────┘└───┘

In [5]:
transpiled = transpile(qc, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |----------------------------------------|
q_0: | U2(0,pi)[160] | DELAY[4340] | CX[1408] |
     |----------------------------------------|
q_1: | DELAY[4500]                 | CX[1408] |
     |----------------------------------------|


###  ASAP or ALAP Schedule

In [6]:
qc = QuantumCircuit(2, name="h2")
qc.h(0)
qc.x(1)
print(qc.draw())
dag = circuit_to_dag(transpile(qc, backend=backend, optimization_level=0))

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘


In [7]:
#ASAP
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |----------------------------|
q_0: | U2(0,pi)[160] | DELAY[160] |
     |----------------------------|
q_1: | U3(pi,0,pi)[320]           |
     |----------------------------|


In [8]:
#ALAP
dag_with_delays = ALAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |----------------------------|
q_0: | DELAY[160] | U2(0,pi)[160] |
     |----------------------------|
q_1: | U3(pi,0,pi)[320]           |
     |----------------------------|


###  Schedule passes convert the duration units into dt (values are rounded)

In [9]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_ns")
bell.h(0)
bell.delay(999, 1, unit='ns')
bell.cx(0,1)
print(bell)
transpiled = transpile(bell, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

       ┌───┐       
q_0: ──┤ H ├────■──
     ┌─┴───┴─┐┌─┴─┐
q_1: ┤ DELAY ├┤ X ├
     └───────┘└───┘
     |----------------------------------------|
q_0: | U2(0,pi)[160] | DELAY[4336] | CX[1408] |
     |----------------------------------------|
q_1: | DELAY[4496]                 | CX[1408] |
     |----------------------------------------|


/Users/itoko/swd/qiskit-dev/qiskit-sdk-py/qiskit/transpiler/passes/scheduling/asap.py:51: UserWarning: Duration of delay is rounded to 4496 dt = 9.991111e-07 s from 9.990000e-07
  UserWarning)


In [10]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_unitless")
bell.h(0)
bell.delay(999, 1)
bell.cx(0,1)
print(bell)
transpiled = transpile(bell, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

       ┌───┐       
q_0: ──┤ H ├────■──
     ┌─┴───┴─┐┌─┴─┐
q_1: ┤ DELAY ├┤ X ├
     └───────┘└───┘
     |---------------------------------------|
q_0: | U2(0,pi)[160] | DELAY[839] | CX[1408] |
     |---------------------------------------|
q_1: | DELAY[999]                 | CX[1408] |
     |---------------------------------------|
